# Model fitting and experiment listing

This notebook shows how we can train models and list previously trained ones (along with useful information about them).

**Important:** this is just a demo to show how models are trained, due to the way Edward manages Tensorflow sessions,
each model should be trained in a different Python session, otherwise restoring it (notebooks 4-1 and 4-2 will throw errors.

In [2]:
import os

import numpy as np

from neural_clustering.model import dpmm, gmm, util
from neural_clustering.criticize import summarize_experiments
from neural_clustering import config

/Users/Edu/miniconda3/envs/test-project/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
cfg = config.load('../config.yaml')

## Model training

### Load training data (generated in notebook 2)

In [5]:
x_train = np.load(os.path.join(cfg['root'], 'training.npy'))
print(f'x_train shape: {x_train.shape}')

x_train shape: (1876, 21)


### ...or generate toy dataset

In [6]:
# x_train = util.build_toy_dataset(500)
# print(f'x_train shape: {x_train.shape}')

## (truncated) Dirichlet Process Mixture Model

$$p(x_n | \pi, \mu, \sigma) = \sum_{k=1}^{K} \pi_k \mathrm{Normal}(x_n |\; \mu_k, \sigma_k)$$

$$ \beta_k \sim \mathrm{Beta}(1,\alpha) $$

$$ \pi_i = \beta_i \prod_{j=1}^{i-1}(1-\beta_j) $$

$$\mu_k \sim \mathrm{Normal} (\mu_k |\; \mathbf{0}, \mathbf{I}) $$

$$\sigma_k^2 \sim \mathrm{Gamma}(\sigma^2_k |\; a, b) $$

There is a bug in Edward that *sometimes* throws an error when fitting, it
occurs more often with truncation levels > 5. It has been reported in [discourse](https://discourse.edwardlib.org/t/variational-inference-for-dirichlet-process-mixtures/251/2) already

In [14]:
# small number of iterations for testing
dpmm.fit(x_train, truncation_level=3, cfg=cfg,
         inference_params=dict(n_iter=500))

500/500 [100%] ██████████████████████████████ Elapsed: 6s | Loss: 512528.469


INFO:neural_clustering.model.dpmm:Session saved in /Users/Edu/data/neural-clustering/sessions/30-Nov-2017@15-18-08-DPMM/session.ckpt
INFO:neural_clustering.model.dpmm:Training data saved in /Users/Edu/data/neural-clustering/sessions/30-Nov-2017@15-18-08-DPMM/training.npy
INFO:neural_clustering.model.dpmm:Params saved in /Users/Edu/data/neural-clustering/sessions/30-Nov-2017@15-18-08-DPMM/params.yaml


## Gaussian Mixture Model

$$p(x_n | \pi, \mu, \sigma) = \sum_{k=1}^{K} \pi_k \mathrm{Normal}(x_n |\; \mu_k, \sigma_k)$$

$$\pi \sim \mathrm{Dirichlet}(\pi, \alpha \mathbf{1}_K) $$

$$\mu_k \sim \mathrm{Normal} (\mu_k |\; \mathbf{0}, \mathbf{I}) $$

$$\sigma_k^2 \sim \mathrm{Gamma}(\sigma^2_k |\; a, b) $$

In [8]:
# small number of iterations for testing
gmm.fit(x_train, k=13, cfg=cfg, samples=10)

INFO:neural_clustering.model.gmm:Session saved in /Users/Edu/data/neural-clustering/sessions/30-Nov-2017@16-42-15-GMM/session.ckpt
INFO:neural_clustering.model.gmm:Training data saved in /Users/Edu/data/neural-clustering/sessions/30-Nov-2017@16-42-15-GMM/training.npy
INFO:neural_clustering.model.gmm:Params saved in /Users/Edu/data/neural-clustering/sessions/30-Nov-2017@16-42-15-GMM/params.yaml


## List experiments

List all previously trained models along with some useful information.

In [5]:
summarize_experiments(cfg)

inference_algoritm,truncation_level,git_hash,name,timestamp,k,model_type,samples,inference_params
KLqp,3,476d12b model fixes,30-Nov-2017@17-11-26-DPMM,2017-11-30T17:11:26.232029,,DPMM,,"{'n_iter': 50000, 'n_samples': 3}"
Gibbs,,476d12b model fixes,30-Nov-2017@16-58-38-GMM,2017-11-30T16:58:38.420122,10,GMM,3000,
KLqp,5,476d12b model fixes,30-Nov-2017@17-17-25-DPMM,2017-11-30T17:17:25.008087,,DPMM,,"{'n_iter': 50000, 'n_samples': 3}"
Gibbs,,476d12b model fixes,30-Nov-2017@16-53-17-GMM,2017-11-30T16:53:17.493203,5,GMM,3000,
KLqp,10,476d12b model fixes,30-Nov-2017@17-25-22-DPMM,2017-11-30T17:25:22.180357,,DPMM,,"{'n_iter': 50000, 'n_samples': 3}"
Gibbs,,476d12b model fixes,30-Nov-2017@17-07-40-GMM,2017-11-30T17:07:40.444634,15,GMM,3000,
